In [11]:
# -*- coding: utf-8 -*-
"""
Parabolic Three-Stage Regression with Quadratic CreditScore Features
Target: MSE < 25.00
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# 0. ЗАГРУЗКА И ПРЕДВАРИТЕЛЬНЫЙ АНАЛИЗ
# ============================================================================

print("=" * 60)
print("ПАРАБОЛИЧЕСКАЯ ТРЕХГРУППОВАЯ МОДЕЛЬ")
print("=" * 60)

# Загрузка данных
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
test_ids = test_df['ID'].copy()

# Базовая фильтрация
def filter_risk_score(df):
    if 'RiskScore' not in df.columns:
        return df

    df = df[
        (df['RiskScore'] != -9999999) &
        (df['RiskScore'] != 10000000) &
        (df['RiskScore'] >= 0) &
        (df['RiskScore'] <= 100)
    ].copy()

    return df

train_df = filter_risk_score(train_df)
print(f"После фильтрации: {len(train_df)} записей")

# ============================================================================
# 1. ОПРЕДЕЛЕНИЕ ПАРАБОЛИЧЕСКОЙ ЗАВИСИМОСТИ
# ============================================================================

# Анализируем параболическую зависимость
x = train_df['CreditScore'].values
y = train_df['RiskScore'].values
valid_mask = ~np.isnan(x) & ~np.isnan(y)
x_valid = x[valid_mask]
y_valid = y[valid_mask]

# Находим параметры параболы
coeffs = np.polyfit(x_valid, y_valid, 2)  # a, b, c для ax² + bx + c
a, b, c = coeffs
vertex_x = -b / (2 * a)  # Вершина параболы
vertex_y = a * vertex_x**2 + b * vertex_x + c

print(f"\n📊 Параметры параболы:")
print(f"  Уравнение: y = {a:.6f}x² + {b:.4f}x + {c:.2f}")
print(f"  Вершина: x = {vertex_x:.1f}, y = {vertex_y:.1f}")

# Определяем границы групп на основе анализа данных
# Анализируем распределение CreditScore
credit_score_counts = train_df['CreditScore'].value_counts()
print(f"\n📊 Анализ распределения CreditScore:")
print(f"  Минимум: {train_df['CreditScore'].min()}")
print(f"  Максимум: {train_df['CreditScore'].max()}")
print(f"  Медиана: {train_df['CreditScore'].median()}")
print(f"  25-й перцентиль: {train_df['CreditScore'].quantile(0.25)}")
print(f"  75-й перцентиль: {train_df['CreditScore'].quantile(0.75)}")

# Определяем границы групп на основе квартилей
q25 = train_df['CreditScore'].quantile(0.25)
q75 = train_df['CreditScore'].quantile(0.75)

# Устанавливаем границы групп
group1_boundary = 600  # Более консервативная граница
group2_boundary = 750  # Другая граница для лучшего баланса

print(f"\n🎯 Границы групп:")
print(f"  Группа 1 (низкие): CreditScore ≤ {group1_boundary}")
print(f"  Группа 2 (средние): {group1_boundary} < CreditScore ≤ {group2_boundary}")
print(f"  Группа 3 (высокие): CreditScore > {group2_boundary}")

# ============================================================================
# 2. РАЗБИЕНИЕ НА ТРИ ГРУППЫ
# ============================================================================

y_all = train_df['RiskScore'].values
train_features = train_df.drop('RiskScore', axis=1)

# Определяем маски для трех групп
mask_low = (train_features['CreditScore'] <= group1_boundary)
mask_mid = (train_features['CreditScore'] > group1_boundary) & (train_features['CreditScore'] <= group2_boundary)
mask_high = (train_features['CreditScore'] > group2_boundary)

# Тренировочные данные по группам
train_low = train_features[mask_low].copy()
train_mid = train_features[mask_mid].copy()
train_high = train_features[mask_high].copy()

y_low = y_all[mask_low]
y_mid = y_all[mask_mid]
y_high = y_all[mask_high]

# Тестовые данные по группам
test_low = test_df[test_df['CreditScore'] <= group1_boundary].copy()
test_mid = test_df[(test_df['CreditScore'] > group1_boundary) & (test_df['CreditScore'] <= group2_boundary)].copy()
test_high = test_df[test_df['CreditScore'] > group2_boundary].copy()

print(f"\n📊 Распределение данных:")
print(f"  Группа низкие: {len(train_low)} train, {len(test_low)} test")
print(f"  Группа средние: {len(train_mid)} train, {len(test_mid)} test")
print(f"  Группа высокие: {len(train_high)} train, {len(test_high)} test")

print(f"\n📊 Средний RiskScore:")
print(f"  Группа низкие: {y_low.mean():.2f} ± {y_low.std():.2f}")
print(f"  Группа средние: {y_mid.mean():.2f} ± {y_mid.std():.2f}")
print(f"  Группа высокие: {y_high.mean():.2f} ± {y_high.std():.2f}")

# ============================================================================
# 3. ФУНКЦИЯ ПРЕДОБРАБОТКИ С КВАДРАТИЧНЫМИ ПРИЗНАКАМИ
# ============================================================================

def parabolic_preprocessing(df_train, df_test, y_train=None, group_name='mid'):
    """
    Предобработка с квадратичными преобразованиями для параболической зависимости
    """
    df_train = df_train.copy()
    df_test = df_test.copy()

    # Сохраняем ID теста
    test_id = None
    if 'ID' in df_test.columns:
        test_id = df_test['ID'].copy()

    # Удаляем ID
    if 'ID' in df_train.columns:
        df_train = df_train.drop('ID', axis=1)
    if 'ID' in df_test.columns:
        df_test = df_test.drop('ID', axis=1)

    # 1. УДАЛЯЕМ ПРОБЛЕМНЫЕ ПРИЗНАКИ
    to_drop = [
        'ApplicationDate', 'MonthlyLoanPayment', 'InterestRate',
        'BaseInterestRate', 'TotalDebtToIncomeRatio',
        'NetWorth', 'TotalAssets', 'MonthlyIncome'
    ]

    for col in to_drop:
        if col in df_train.columns:
            df_train = df_train.drop(col, axis=1)
        if col in df_test.columns:
            df_test = df_test.drop(col, axis=1)

    # 2. КЛЮЧЕВЫЕ КВАДРАТИЧНЫЕ ПРЕОБРАЗОВАНИЯ

    # CreditScore квадратичные преобразования
    if 'CreditScore' in df_train.columns:
        # Основное квадратичное преобразование
        df_train['CreditScore_squared'] = df_train['CreditScore'] ** 2
        df_test['CreditScore_squared'] = df_test['CreditScore'] ** 2

        # Взаимодействие с другими признаками
        if 'DebtToIncomeRatio' in df_train.columns:
            df_train['CreditScore_x_DTI'] = df_train['CreditScore'] * df_train['DebtToIncomeRatio']
            df_test['CreditScore_x_DTI'] = df_test['CreditScore'] * df_test['DebtToIncomeRatio']

            df_train['CreditScore_sq_x_DTI'] = df_train['CreditScore_squared'] * df_train['DebtToIncomeRatio']
            df_test['CreditScore_sq_x_DTI'] = df_test['CreditScore_squared'] * df_test['DebtToIncomeRatio']

        # Расстояние до вершины параболы
        df_train['Dist_to_Vertex'] = np.abs(df_train['CreditScore'] - vertex_x)
        df_test['Dist_to_Vertex'] = np.abs(df_test['CreditScore'] - vertex_x)

        # Квадрат расстояния до вершины (параболический штраф)
        df_train['Dist_to_Vertex_sq'] = (df_train['CreditScore'] - vertex_x) ** 2
        df_test['Dist_to_Vertex_sq'] = (df_test['CreditScore'] - vertex_x) ** 2

    # Квадратичные преобразования для других числовых признаков
    numeric_features = ['Age', 'AnnualIncome', 'LoanAmount', 'DebtToIncomeRatio',
                       'NumberOfOpenCreditLines', 'LengthOfCreditHistory']

    for feature in numeric_features:
        if feature in df_train.columns:
            # Квадрат
            df_train[f'{feature}_squared'] = df_train[feature] ** 2
            df_test[f'{feature}_squared'] = df_test[feature] ** 2

    # Логарифмические преобразования для skewed features
    skewed_features = ['AnnualIncome', 'LoanAmount', 'TotalLiabilities']
    for feature in skewed_features:
        if feature in df_train.columns:
            df_train[f'log_{feature}'] = np.log1p(np.abs(df_train[feature]) + 1)
            df_test[f'log_{feature}'] = np.log1p(np.abs(df_test[feature]) + 1)

    # 3. ФИНАНСОВЫЕ СООТНОШЕНИЯ

    if 'AnnualIncome' in df_train.columns and 'LoanAmount' in df_train.columns:
        df_train['IncomeToLoan'] = df_train['AnnualIncome'] / (df_train['LoanAmount'] + 1)
        df_test['IncomeToLoan'] = df_test['AnnualIncome'] / (df_test['LoanAmount'] + 1)

        # Квадрат соотношения
        df_train['IncomeToLoan_sq'] = df_train['IncomeToLoan'] ** 2
        df_test['IncomeToLoan_sq'] = df_test['IncomeToLoan'] ** 2

    if 'TotalLiabilities' in df_train.columns and 'AnnualIncome' in df_train.columns:
        df_train['LiabilitiesToIncome'] = df_train['TotalLiabilities'] / (df_train['AnnualIncome'] + 1)
        df_test['LiabilitiesToIncome'] = df_test['TotalLiabilities'] / (df_test['AnnualIncome'] + 1)

    # 4. ФЛАГИ И БИНАРНЫЕ ПРИЗНАКИ

    if 'DebtToIncomeRatio' in df_train.columns:
        df_train['HighDebt'] = (df_train['DebtToIncomeRatio'] > 0.4).astype(int)
        df_test['HighDebt'] = (df_test['DebtToIncomeRatio'] > 0.4).astype(int)

    if 'CreditScore' in df_train.columns:
        df_train['CreditScore_IsMax'] = (df_train['CreditScore'] == 850).astype(int)
        df_test['CreditScore_IsMax'] = (df_test['CreditScore'] == 850).astype(int)

        df_train['CreditScore_IsMin'] = (df_train['CreditScore'] == 300).astype(int)
        df_test['CreditScore_IsMin'] = (df_test['CreditScore'] == 300).astype(int)

    # 5. ОБРАБОТКА ПРОПУСКОВ

    for col in df_train.columns:
        if df_train[col].dtype in [np.float64, np.int64]:
            median_val = df_train[col].median()
            df_train[col] = df_train[col].fillna(median_val)
            df_test[col] = df_test[col].fillna(median_val)
        else:
            mode_val = df_train[col].mode()
            fill_val = mode_val[0] if not mode_val.empty else 'Unknown'
            df_train[col] = df_train[col].fillna(fill_val)
            df_test[col] = df_test[col].fillna(fill_val)

    # 6. ОБРАБОТКА ВЫБРОСОВ (winsorizing)
    winzorize_cols = ['AnnualIncome', 'DebtToIncomeRatio', 'LoanAmount']
    for col in winzorize_cols:
        if col in df_train.columns:
            df_train[col] = stats.mstats.winsorize(df_train[col].values, limits=[0.03, 0.03])
            df_test[col] = stats.mstats.winsorize(df_test[col].values, limits=[0.03, 0.03])

    # 7. ONE-HOT ENCODING

    cat_cols = df_train.select_dtypes(include=['object']).columns.tolist()
    if cat_cols:
        df_train = pd.get_dummies(df_train, columns=cat_cols, drop_first=True)
        df_test = pd.get_dummies(df_test, columns=cat_cols, drop_first=True)

    # 8. ВЫРАВНИВАНИЕ ПРИЗНАКОВ

    common_cols = df_train.columns.intersection(df_test.columns)
    df_train = df_train[common_cols]
    df_test = df_test[common_cols]

    for col in df_train.columns:
        if col not in df_test.columns:
            df_test[col] = 0

    df_test = df_test[df_train.columns]

    # 9. ПРЕОБРАЗОВАНИЕ И МАСШТАБИРОВАНИЕ

    X_train = df_train.values.astype(np.float64)
    X_test = df_test.values.astype(np.float64)

    X_train = np.nan_to_num(X_train, nan=0.0, posinf=1e10, neginf=-1e10)
    X_test = np.nan_to_num(X_test, nan=0.0, posinf=1e10, neginf=-1e10)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    print(f"  Группа {group_name}: {X_train_scaled.shape[1]} признаков")

    return X_train_scaled, X_test_scaled, test_id

# ============================================================================
# 4. ПРЕДОБРАБОТКА ДЛЯ КАЖДОЙ ГРУППЫ
# ============================================================================

print("\n" + "=" * 60)
print("ПРЕДОБРАБОТКА ДАННЫХ")
print("=" * 60)

print("\nОбработка группы: низкие (CreditScore ≤ {})".format(group1_boundary))
X_low_train, X_low_test, test_ids_low = parabolic_preprocessing(
    train_low, test_low, y_low, group_name='низкие'
)

print("\nОбработка группы: средние ({} < CreditScore ≤ {})".format(group1_boundary, group2_boundary))
X_mid_train, X_mid_test, test_ids_mid = parabolic_preprocessing(
    train_mid, test_mid, y_mid, group_name='средние'
)

print("\nОбработка группы: высокие (CreditScore > {})".format(group2_boundary))
X_high_train, X_high_test, test_ids_high = parabolic_preprocessing(
    train_high, test_high, y_high, group_name='высокие'
)

# ============================================================================
# 5. ИСПРАВЛЕННАЯ ФУНКЦИЯ ДЛЯ ОТБОРА ПРИЗНАКОВ И КРОСС-ВАЛИДАЦИИ
# ============================================================================

def train_with_cross_val(X_train, y_train, X_test, group_name, n_features=40):
    """
    Обучение с кросс-валидацией и отбором признаков
    """
    # 1. Отбор признаков по корреляции
    correlations = []
    for i in range(X_train.shape[1]):
        corr = np.corrcoef(X_train[:, i], y_train)[0, 1]
        correlations.append(abs(corr))

    # Отбираем топ-n_features признаков
    top_idx = np.argsort(correlations)[-n_features:]
    X_train_sel = X_train[:, top_idx]

    # Подготовка X_test_sel (может быть пустым)
    if len(X_test) > 0:
        X_test_sel = X_test[:, top_idx]
    else:
        X_test_sel = np.array([]).reshape(0, n_features)  # Создаем пустой массив

    # 2. Кросс-валидация для подбора alpha
    best_alpha = None
    best_mse = float('inf')
    all_scores = []

    # Разные диапазоны alpha для разных групп
    if group_name == 'низкие':
        alphas = [0.1, 0.5, 1, 2, 5, 10, 20, 50, 100]
    elif group_name == 'средние':
        alphas = [0.01, 0.1, 0.5, 1, 2, 5, 10, 20, 50]
    else:  # высокие
        alphas = [0.5, 1, 2, 5, 10, 20, 50, 100]

    kf = KFold(n_splits=min(5, len(X_train_sel)), shuffle=True, random_state=42)

    for alpha in alphas:
        fold_scores = []

        for train_idx, val_idx in kf.split(X_train_sel):
            X_tr, X_val = X_train_sel[train_idx], X_train_sel[val_idx]
            y_tr, y_val = y_train[train_idx], y_val[val_idx]

            model = Ridge(alpha=alpha, random_state=42)
            model.fit(X_tr, y_tr)

            y_pred = model.predict(X_val)
            mse = np.mean((y_val - y_pred) ** 2)
            fold_scores.append(mse)

        avg_mse = np.mean(fold_scores)
        all_scores.append((alpha, avg_mse))

        if avg_mse < best_mse:
            best_mse = avg_mse
            best_alpha = alpha

    # 3. Обучение финальной модели
    final_model = Ridge(alpha=best_alpha, random_state=42)
    final_model.fit(X_train_sel, y_train)

    # 4. Предсказания
    train_pred = final_model.predict(X_train_sel)
    train_mse = np.mean((y_train - train_pred) ** 2)

    if len(X_test_sel) > 0:
        test_pred = final_model.predict(X_test_sel)
    else:
        test_pred = np.array([])

    print(f"  {group_name}: alpha={best_alpha}, CV MSE={best_mse:.2f}, Train MSE={train_mse:.2f}")

    return final_model, X_test_sel, test_pred, best_mse, top_idx

# ============================================================================
# 6. ОБУЧЕНИЕ МОДЕЛЕЙ ДЛЯ КАЖДОЙ ГРУППЫ (С ЗАЩИТОЙ ОТ ПУСТЫХ ДАННЫХ)
# ============================================================================

print("\n" + "=" * 60)
print("ОБУЧЕНИЕ МОДЕЛЕЙ С КРОСС-ВАЛИДАЦИЕЙ")
print("=" * 60)

models = {}
predictions = {}
cv_mses = {}

# Группа низкие (только если есть данные)
if len(train_low) > 0:
    print(f"\n🎯 Обучение группы 'низкие' (n={len(train_low)})")
    if len(train_low) >= 10:  # Минимум 10 записей для кросс-валидации
        model_low, X_low_test_sel, pred_low, mse_low, idx_low = train_with_cross_val(
            X_low_train, y_low, X_low_test, 'низкие', n_features=min(35, X_low_train.shape[1])
        )
        models['low'] = model_low
        predictions['low'] = pred_low
        cv_mses['low'] = mse_low
    else:
        print("  ⚠️  Слишком мало данных для кросс-валидации")
        # Используем простую модель
        model_low = Ridge(alpha=10.0, random_state=42)
        model_low.fit(X_low_train, y_low)
        models['low'] = model_low
        if len(X_low_test) > 0:
            predictions['low'] = model_low.predict(X_low_test)
        else:
            predictions['low'] = np.array([])
        cv_mses['low'] = np.mean((y_low - model_low.predict(X_low_train)) ** 2)
        print(f"  Простая модель: alpha=10.0, Train MSE={cv_mses['low']:.2f}")
else:
    print("  ⚠️  Нет данных в группе 'низкие'")

# Группа средние (основная группа)
print(f"\n🎯 Обучение группы 'средние' (n={len(train_mid)})")
model_mid, X_mid_test_sel, pred_mid, mse_mid, idx_mid = train_with_cross_val(
    X_mid_train, y_mid, X_mid_test, 'средние', n_features=min(40, X_mid_train.shape[1])
)
models['mid'] = model_mid
predictions['mid'] = pred_mid
cv_mses['mid'] = mse_mid

# Группа высокие (только если есть данные)
if len(train_high) > 0:
    print(f"\n🎯 Обучение группы 'высокие' (n={len(train_high)})")
    if len(train_high) >= 10:
        model_high, X_high_test_sel, pred_high, mse_high, idx_high = train_with_cross_val(
            X_high_train, y_high, X_high_test, 'высокие', n_features=min(35, X_high_train.shape[1])
        )
        models['high'] = model_high
        predictions['high'] = pred_high
        cv_mses['high'] = mse_high
    else:
        print("  ⚠️  Слишком мало данных для кросс-валидации")
        model_high = Ridge(alpha=5.0, random_state=42)
        model_high.fit(X_high_train, y_high)
        models['high'] = model_high
        if len(X_high_test) > 0:
            predictions['high'] = model_high.predict(X_high_test)
        else:
            predictions['high'] = np.array([])
        cv_mses['high'] = np.mean((y_high - model_high.predict(X_high_train)) ** 2)
        print(f"  Простая модель: alpha=5.0, Train MSE={cv_mses['high']:.2f}")
else:
    print("  ⚠️  Нет данных в группе 'высокие'")

# ============================================================================
# 7. СБОРКА ФИНАЛЬНЫХ ПРЕДСКАЗАНИЙ
# ============================================================================

print("\n" + "=" * 60)
print("СБОРКА ПРЕДСКАЗАНИЙ")
print("=" * 60)

# Создаем массив для предсказаний
y_test_pred = np.zeros(len(test_df))

# Определяем индексы для каждой группы в тестовых данных
idx_test_low = test_df[test_df['CreditScore'] <= group1_boundary].index
idx_test_mid = test_df[(test_df['CreditScore'] > group1_boundary) &
                       (test_df['CreditScore'] <= group2_boundary)].index
idx_test_high = test_df[test_df['CreditScore'] > group2_boundary].index

# Заполняем предсказания с проверкой
print("\n📊 Заполнение предсказаний по группам:")

# Группа низкие
if 'low' in predictions and len(predictions['low']) > 0:
    for i, idx in enumerate(idx_test_low):
        if i < len(predictions['low']):
            y_test_pred[idx] = predictions['low'][i]
    print(f"  Группа низкие: {len(predictions['low'])} предсказаний")
elif len(idx_test_low) > 0:
    print(f"  ⚠️  Группа низкие: нет предсказаний, заполняем средним")
    # Используем среднее значение из тренировочных данных
    mean_low = y_low.mean() if len(y_low) > 0 else 50.0
    for idx in idx_test_low:
        y_test_pred[idx] = mean_low

# Группа средние
if 'mid' in predictions and len(predictions['mid']) > 0:
    for i, idx in enumerate(idx_test_mid):
        if i < len(predictions['mid']):
            y_test_pred[idx] = predictions['mid'][i]
    print(f"  Группа средние: {len(predictions['mid'])} предсказаний")

# Группа высокие
if 'high' in predictions and len(predictions['high']) > 0:
    for i, idx in enumerate(idx_test_high):
        if i < len(predictions['high']):
            y_test_pred[idx] = predictions['high'][i]
    print(f"  Группа высокие: {len(predictions['high'])} предсказаний")
elif len(idx_test_high) > 0:
    print(f"  ⚠️  Группа высокие: нет предсказаний, заполняем средним")
    # Используем среднее значение из тренировочных данных
    mean_high = y_high.mean() if len(y_high) > 0 else 50.0
    for idx in idx_test_high:
        y_test_pred[idx] = mean_high

# Обработка пропущенных значений
missing_mask = (y_test_pred == 0)
if missing_mask.any():
    print(f"  ⚠️  Найдено {missing_mask.sum()} пропущенных значений")

    # Используем параболическую модель для заполнения
    for idx in np.where(missing_mask)[0]:
        cs = test_df.loc[idx, 'CreditScore']
        # Параболическое предсказание
        y_para = a * cs**2 + b * cs + c
        y_test_pred[idx] = np.clip(y_para, 0, 100)

# Постобработка
y_test_pred = np.clip(y_test_pred, 0, 100)
y_test_pred = np.round(y_test_pred, 1)

# ============================================================================
# 8. АНАЛИЗ РЕЗУЛЬТАТОВ
# ============================================================================

print("\n" + "=" * 60)
print("АНАЛИЗ РЕЗУЛЬТАТОВ")
print("=" * 60)

print(f"\n📊 Статистика предсказаний:")
print(f"  Минимум: {y_test_pred.min():.1f}")
print(f"  Максимум: {y_test_pred.max():.1f}")
print(f"  Среднее: {y_test_pred.mean():.1f}")
print(f"  Медиана: {np.median(y_test_pred):.1f}")
print(f"  Стандартное отклонение: {y_test_pred.std():.1f}")

print(f"\n📊 Результаты кросс-валидации:")
for group_name, mse in cv_mses.items():
    if group_name == 'low':
        group_label = 'низкие'
    elif group_name == 'mid':
        group_label = 'средние'
    else:
        group_label = 'высокие'
    print(f"  Группа {group_label}: MSE = {mse:.2f}")

# ============================================================================
# 9. СОХРАНЕНИЕ РЕЗУЛЬТАТОВ
# ============================================================================

print("\n" + "=" * 60)
print("СОХРАНЕНИЕ РЕЗУЛЬТАТОВ")
print("=" * 60)

# Создаем submission файл
submission = pd.DataFrame({
    'ID': test_ids,
    'RiskScore': y_test_pred
})

# Проверяем корректность
print(f"\n✅ Проверка submission:")
print(f"  Всего записей: {len(submission)}")
print(f"  NaN значений: {submission['RiskScore'].isnull().sum()}")
print(f"  Вне диапазона [0, 100]: {((submission['RiskScore'] < 0) | (submission['RiskScore'] > 100)).sum()}")

# Сохраняем
submission.to_csv('parabolic_three_groups_fixed_submission.csv', index=False)
print(f"\n✅ Результаты сохранены в parabolic_three_groups_fixed_submission.csv")
print(f"\nПервые 10 предсказаний:")
print(submission.head(10))

# ============================================================================
# 10. ВИЗУАЛИЗАЦИЯ РЕЗУЛЬТАТОВ
# ============================================================================

# Визуализация результатов
plt.figure(figsize=(15, 5))

# 1. Распределение предсказаний
plt.subplot(1, 3, 1)
plt.hist(y_test_pred, bins=30, color='purple', edgecolor='black', alpha=0.7)
plt.xlabel('Predicted RiskScore')
plt.ylabel('Частота')
plt.title('Распределение предсказанных RiskScore')
plt.grid(True, alpha=0.3)

# 2. Параболическая зависимость
plt.subplot(1, 3, 2)
plt.scatter(train_df['CreditScore'], train_df['RiskScore'], alpha=0.1, s=5, color='blue', label='Train data')

# Добавляем предсказания (если не слишком много)
if len(test_df) < 1000:
    plt.scatter(test_df['CreditScore'], y_test_pred, alpha=0.3, s=10, color='red', label='Predictions')

# Параболическая кривая
x_range = np.linspace(300, 850, 100)
y_range = a * x_range**2 + b * x_range + c
plt.plot(x_range, y_range, 'g-', linewidth=2, label='Парабола')

# Границы групп
plt.axvline(x=group1_boundary, color='orange', linestyle='--', label=f'Граница 1 ({group1_boundary})')
plt.axvline(x=group2_boundary, color='brown', linestyle='--', label=f'Граница 2 ({group2_boundary})')

plt.xlabel('CreditScore')
plt.ylabel('RiskScore')
plt.title('Параболическая модель и предсказания')
plt.legend()
plt.grid(True, alpha=0.3)

# 3. MSE по группам
plt.subplot(1, 3, 3)
group_labels = []
mse_values = []

for group_name, mse in cv_mses.items():
    if group_name == 'low':
        group_labels.append('Низкие')
    elif group_name == 'mid':
        group_labels.append('Средние')
    else:
        group_labels.append('Высокие')
    mse_values.append(mse)

if mse_values:
    colors = ['red' if mse > 25 else 'green' for mse in mse_values]
    bars = plt.bar(group_labels, mse_values, color=colors, alpha=0.7)
    plt.axhline(y=25, color='black', linestyle='--', label='Цель (MSE=25)')
    plt.ylabel('MSE')
    plt.title('MSE по группам')
    plt.grid(True, alpha=0.3, axis='y')

    # Добавляем значения на столбцы
    for bar, mse in zip(bars, mse_values):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{mse:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print(f"\n{'='*60}")
print("ПАРАБОЛИЧЕСКАЯ ТРЕХГРУППОВАЯ МОДЕЛЬ ЗАВЕРШЕНА!")
print(f"{'='*60}")

ПАРАБОЛИЧЕСКАЯ ТРЕХГРУППОВАЯ МОДЕЛЬ
После фильтрации: 10272 записей

📊 Параметры параболы:
  Уравнение: y = -0.000064x² + 0.0017x + 78.61
  Вершина: x = 13.5, y = 78.6

📊 Анализ распределения CreditScore:
  Минимум: 300.0
  Максимум: 850.0
  Медиана: 722.0
  25-й перцентиль: 550.0
  75-й перцентиль: 850.0

🎯 Границы групп:
  Группа 1 (низкие): CreditScore ≤ 600
  Группа 2 (средние): 600 < CreditScore ≤ 750
  Группа 3 (высокие): CreditScore > 750

📊 Распределение данных:
  Группа низкие: 3095 train, 1574 test
  Группа средние: 2248 train, 1114 test
  Группа высокие: 4444 train, 2312 test

📊 Средний RiskScore:
  Группа низкие: 66.96 ± 10.55
  Группа средние: 47.78 ± 13.29
  Группа высокие: 35.67 ± 9.31

ПРЕДОБРАБОТКА ДАННЫХ

Обработка группы: низкие (CreditScore ≤ 600)
  Группа низкие: 57 признаков

Обработка группы: средние (600 < CreditScore ≤ 750)
  Группа средние: 57 признаков

Обработка группы: высокие (CreditScore > 750)
  Группа высокие: 57 признаков

ОБУЧЕНИЕ МОДЕЛЕЙ С КРОСС-ВАЛИ

UnboundLocalError: cannot access local variable 'y_val' where it is not associated with a value